# Database Processes

**Objective:** Sample code for generic database operations.

**Author:** Phil Busko

In [1]:
import os, sys
import pandas as PD

In [2]:
MODULE_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(MODULE_PATH, 'data')
LOGIC_PATH = os.path.join(MODULE_PATH, 'logic')
sys.path.append(LOGIC_PATH)
print(MODULE_PATH)

C:\Users\pbusko\Projects\QualityCuration\base_module


In [3]:
import sqlalchemy as SQL
import database_helper as DB
print([x for x in dir(DB) if '_' not in x and len(x) > 8])
dbPath = os.path.join(DATA_PATH, 'base.db')
meta, engine = DB.openConnection(dbPath)

['openConnection']
sqlite:///C:\Users\pbusko\Projects\QualityCuration\base_module\data\base.db


In [4]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import IPython.display as DS 
from ipywidgets import IntProgress

### Load From File 

In [5]:
brickFile = os.path.join(DATA_PATH, 'brickset_set_filter.csv')
brickDf = PD.read_csv(brickFile)
brickDf.head(3)

,set_no,name,price_store,price_new,price_used,rating_value,rating_votes,theme_group,theme,subtheme,main_tag,year,volume,weight,piece_cnt,minifig_cnt
0,858-1,Auto Engines,NaN,NaN,51.0,4.5,2.0,Technical,Technic,NaN,NaN,1980.0,NaN,NaN,242.0,0.0
1,1591-1,Danone Delivery Truck,NaN,129.0,28.0,NaN,NaN,Modern day,Town,Special,NaN,1980.0,NaN,NaN,40.0,0.0
2,1592-1,Town Square - Castle Scene,NaN,1168.0,191.0,4.8,4.0,Modern day,Town,Classic,NaN,1980.0,NaN,NaN,471.0,11.0


In [6]:
brickDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6181 entries, 0 to 6180
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   set_no        6181 non-null   object 
 1   name          6181 non-null   object 
 2   price_store   4767 non-null   float64
 3   price_new     5780 non-null   float64
 4   price_used    5489 non-null   float64
 5   rating_value  5005 non-null   float64
 6   rating_votes  5005 non-null   float64
 7   theme_group   6181 non-null   object 
 8   theme         6181 non-null   object 
 9   subtheme      4494 non-null   object 
 10  main_tag      0 non-null      float64
 11  year          6181 non-null   float64
 12  volume        2418 non-null   float64
 13  weight        2453 non-null   float64
 14  piece_cnt     6137 non-null   float64
 15  minifig_cnt   6181 non-null   float64
dtypes: float64(11), object(5)
memory usage: 772.8+ KB


### Create Table

In [7]:
# get existing table names
# reflect the meta again to refresh any deleted tables

meta.clear()
meta.reflect()

existingTables = list(meta.tables.keys())
print(existingTables)

['legoSets']


In [8]:
# create table if it doesn't already exist

tableName = 'legoSets'

if tableName not in existingTables:
    setsTb = SQL.Table(tableName, meta, 
        SQL.Column('id', SQL.Integer, primary_key = True), 
        SQL.Column('setNo', SQL.String), 
        SQL.Column('name', SQL.String),
        SQL.Column('theme', SQL.String),
        SQL.Column('pieceCnt', SQL.Integer),
        SQL.UniqueConstraint('setNo', name='setNo_unique')
    )
    meta.create_all(engine)
    print(f"table {tableName} created")
    
else:
    print(f"table {tableName} already exists")

table legoSets already exists


In [9]:
# delete the table when needed
# delete happens on meta after the kernel is restarted
# remove all records first so notebook doesn't freeze

# deleteTable = meta.tables.get(tableName)

# try:
#     statement = deleteTable.delete()
#     results = statement.execute()
#     print(f"rows deleted: {results.rowcount:,}")
#     deleteTable.drop(engine)
#     print("table deleted")
# except Exception as ex:
#     print(ex)

### CRUD Operations

In [10]:
# sqlalchemy core has 2 syntax styles
# SQL.select() is the standard, will work with joins?
# table.select() is syntactic sugar, not as useful
# sessions don't seem to be worth it, they just make it more complicated

In [11]:
# convert dataframe to input type accepted by sqlalchemy insert
# no need to drop extra file columns

convertDf = brickDf.rename(columns={'set_no' : 'setNo', 'piece_cnt' : 'pieceCnt'})
convertLs = convertDf.to_dict(orient='records')
print(*convertLs[:2], sep='\n')

{'setNo': '858-1', 'name': 'Auto Engines', 'price_store': nan, 'price_new': nan, 'price_used': 51.0, 'rating_value': 4.5, 'rating_votes': 2.0, 'theme_group': 'Technical', 'theme': 'Technic', 'subtheme': nan, 'main_tag': nan, 'year': 1980.0, 'volume': nan, 'weight': nan, 'pieceCnt': 242.0, 'minifig_cnt': 0.0}
{'setNo': '1591-1', 'name': 'Danone Delivery Truck', 'price_store': nan, 'price_new': 129.0, 'price_used': 28.0, 'rating_value': nan, 'rating_votes': nan, 'theme_group': 'Modern day', 'theme': 'Town', 'subtheme': 'Special', 'main_tag': nan, 'year': 1980.0, 'volume': nan, 'weight': nan, 'pieceCnt': 40.0, 'minifig_cnt': 0.0}


In [12]:
# bulk insert all data from the file
# sqlalchemy doesn't support de-duplicates in bulk insert, must insert row by row

setsTb = meta.tables.get(tableName)
try:
    results = setsTb.insert().execute(convertLs)
    print(f"rows inserted: {results.rowcount:,}")
except Exception as ex:
    print(ex)

(sqlite3.IntegrityError) UNIQUE constraint failed: legoSets.setNo
[SQL: INSERT INTO "legoSets" ("setNo", name, theme, "pieceCnt") VALUES (?, ?, ?, ?)]
[parameters: (('858-1', 'Auto Engines', 'Technic', 242.0), ('1591-1', 'Danone Delivery Truck', 'Town', 40.0), ('1592-1', 'Town Square - Castle Scene', 'Town', 471.0), ('1651-2', 'Maersk Line Container Lorry', 'Town', 305.0), ('3602-1', 'Pram - Lisa Lamb Goes Walking', 'Fabuland', 3.0), ('3604-1', 'Mark Monkey with his Fruit Stall', 'Fabuland', 3.0), ('3605-1', 'Ricky Racoon and his Scooter', 'Fabuland', 1.0), ('3634-1', "Charlie Crow's Carry-All", 'Fabuland', 17.0)  ... displaying 10 of 6181 total bound parameter sets ...  ('88005-1', 'Headlight Set', 'Powered Up', 1.0), ('4000031-1', 'Exclusive T. rex', 'Jurassic World', 70.0))]
(Background on this error at: http://sqlalche.me/e/14/gkpj)


In [13]:
# select as a tuple, default behavior

statement = SQL.select(setsTb)
print(statement)

results = statement.execute().fetchall()
print(*results[:5], sep='\n')

SELECT "legoSets".id, "legoSets"."setNo", "legoSets".name, "legoSets".theme, "legoSets"."pieceCnt" 
FROM "legoSets"
(1, '858-1', 'Auto Engines', 'Technic', 242)
(2, '1591-1', 'Danone Delivery Truck', 'Town', 40)
(3, '1592-1', 'Town Square - Castle Scene', 'Town', 471)
(4, '1651-2', 'Maersk Line Container Lorry', 'Town', 305)
(5, '3602-1', 'Pram - Lisa Lamb Goes Walking', 'Fabuland', 3)


In [14]:
# select as list of dicts, better for pandas

statement = SQL.select(setsTb)
results = statement.execute()
resultsDx = [dict(x) for x in results]
print(*resultsDx[:5], sep='\n')

{'id': 1, 'setNo': '858-1', 'name': 'Auto Engines', 'theme': 'Technic', 'pieceCnt': 242}
{'id': 2, 'setNo': '1591-1', 'name': 'Danone Delivery Truck', 'theme': 'Town', 'pieceCnt': 40}
{'id': 3, 'setNo': '1592-1', 'name': 'Town Square - Castle Scene', 'theme': 'Town', 'pieceCnt': 471}
{'id': 4, 'setNo': '1651-2', 'name': 'Maersk Line Container Lorry', 'theme': 'Town', 'pieceCnt': 305}
{'id': 5, 'setNo': '3602-1', 'name': 'Pram - Lisa Lamb Goes Walking', 'theme': 'Fabuland', 'pieceCnt': 3}


In [15]:
# find all sets with 3-digit set no

statement = SQL.select([setsTb.c.setNo, setsTb.c.name]
        ).where(SQL.func.length(setsTb.c.setNo)==5
        ).order_by(setsTb.c.setNo.desc())
results = statement.execute()
resultsDx = [dict(x) for x in results]
print(f"rows found: {len(resultsDx)}")
print(*resultsDx[:5], sep='\n')

rows found: 7
{'setNo': '858-1', 'name': 'Auto Engines'}
{'setNo': '801-4', 'name': 'Fabuland Characters'}
{'setNo': '611-2', 'name': 'Air Canada Jet Plane'}
{'setNo': '610-2', 'name': 'Rescue Helicopter'}
{'setNo': '107-2', 'name': 'Mail Truck'}


In [16]:
# check rows for update

statement = SQL.select(setsTb
        ).where(setsTb.c.pieceCnt==None)
results = statement.execute()
resultsDx = [dict(x) for x in results]
print(f"rows found: {len(resultsDx)}")
print(*resultsDx[:3], sep='\n')

rows found: 0



In [17]:
# update null values to 0

statement = SQL.update(setsTb
        ).where(setsTb.c.pieceCnt==None
        ).values({'pieceCnt': 0})
results = statement.execute()
print(statement)
print(f"rows updated: {results.rowcount:,}")

statement = SQL.select(setsTb).where(setsTb.c.pieceCnt==0)
results = statement.execute()
resultsDx = [dict(x) for x in results]
print(*resultsDx[:3], sep='\n')

UPDATE "legoSets" SET "pieceCnt"=? WHERE "legoSets"."pieceCnt" IS NULL
rows updated: 0
{'id': 107, 'setNo': '3694-1', 'name': 'Figure collection', 'theme': 'Fabuland', 'pieceCnt': 0}
{'id': 108, 'setNo': '3695-1', 'name': 'Figure collection', 'theme': 'Fabuland', 'pieceCnt': 0}
{'id': 182, 'setNo': '801-4', 'name': 'Fabuland Characters', 'theme': 'Fabuland', 'pieceCnt': 0}


In [18]:
# delete all records in table

# statement = setsTb.delete()
# results = statement.execute()
# print(statement)
# print(f"rows deleted: {results.rowcount:,}")

# results = setsTb.select().execute()
# resultsDx = [dict(x) for x in results]
# print(*resultsDx[:5], sep='\n')

### Aggregate Operations

In [19]:
# count star

statement = SQL.select(SQL.func.count()).select_from(setsTb)
results = statement.execute().scalar()
print(f"rows: {results:,}")

rows: 6,181


In [20]:
# max, min, count per column

statement = SQL.select(SQL.func.max(setsTb.c.pieceCnt))
results = statement.execute().scalar()
print(f"max pieces: {results:,}")

max pieces: 7,541


In [21]:
# group by with count

statement = SQL.select(setsTb.c.theme, SQL.func.count().label('cnt')
                      ).group_by(setsTb.c.theme).order_by(SQL.desc('cnt'))
results = statement.execute()
resultsDx = [dict(x) for x in results]
PD.DataFrame(resultsDx)

,theme,cnt
0,Town,574
1,Star Wars,508
2,City,431
3,Creator,371
4,Technic,353
...,...,...
89,The Powerpuff Girls,2
90,Ghostbusters,2
91,Avatar The Last Airbender,2
92,Powered Up,1


### Advanced Operations

In [22]:
# user defined function for string split
# existing functions: http://docs.sqlalchemy.org/en/rel_1_0/core/functions.html

@SQL.event.listens_for(SQL.engine.Engine, 'connect')
def sqlite_engine_connect(dbapi_conn, connection_record):
    dbapi_conn.create_function('str_split', 1, _str_split)

def _str_split(inputString):
    splitLs = inputString.split('-')
    if len(splitLs) > 1:
        return splitLs[0]
    return ''

In [23]:
# verify the split works 

statement = SQL.select(setsTb.c.setNo, 
                       SQL.func.str_split(setsTb.c.setNo).label('split'))
results = statement.execute()
resultsDx = [dict(x) for x in results]
print(*resultsDx[:5], sep='\n')

{'setNo': '10000-1', 'split': '10000'}
{'setNo': '10001-1', 'split': '10001'}
{'setNo': '10002-1', 'split': '10002'}
{'setNo': '10013-1', 'split': '10013'}
{'setNo': '10014-1', 'split': '10014'}


In [24]:
# do the setno postfixes matter ?
# must join the table with itself, matching on the prefixes only

setsTb2 = SQL.alias(setsTb)

joinStmt = SQL.join(setsTb, setsTb2, 
                    SQL.and_(SQL.func.str_split(setsTb.c.setNo)==SQL.func.str_split(setsTb2.c.setNo),
                         setsTb.c.id!=setsTb2.c.id))
statement = SQL.select(setsTb.c.id, setsTb.c.setNo, 
                       setsTb2.c.id, setsTb2.c.setNo).select_from(joinStmt)

results = statement.execute()
resultsDx = [dict(x) for x in results]
print(f"rows found: {len(resultsDx)}")
print(*resultsDx[:20], sep='\n')

rows found: 390
{'id': 2659, 'setNo': '1270-2', 'id_1': 823, 'setNo_1': '1270-3'}
{'id': 823, 'setNo': '1270-3', 'id_1': 2659, 'setNo_1': '1270-2'}
{'id': 895, 'setNo': '1843-1', 'id_1': 896, 'setNo_1': '1843-2'}
{'id': 896, 'setNo': '1843-2', 'id_1': 895, 'setNo_1': '1843-1'}
{'id': 473, 'setNo': '1974-2', 'id_1': 474, 'setNo_1': '1974-3'}
{'id': 473, 'setNo': '1974-2', 'id_1': 475, 'setNo_1': '1974-4'}
{'id': 474, 'setNo': '1974-3', 'id_1': 473, 'setNo_1': '1974-2'}
{'id': 474, 'setNo': '1974-3', 'id_1': 475, 'setNo_1': '1974-4'}
{'id': 475, 'setNo': '1974-4', 'id_1': 473, 'setNo_1': '1974-2'}
{'id': 475, 'setNo': '1974-4', 'id_1': 474, 'setNo_1': '1974-3'}
{'id': 3391, 'setNo': '21000-1', 'id_1': 3948, 'setNo_1': '21000-2'}
{'id': 3948, 'setNo': '21000-2', 'id_1': 3391, 'setNo_1': '21000-1'}
{'id': 1570, 'setNo': '3401-1', 'id_1': 1571, 'setNo_1': '3401-2'}
{'id': 1571, 'setNo': '3401-2', 'id_1': 1570, 'setNo_1': '3401-1'}
{'id': 1574, 'setNo': '3404-1', 'id_1': 1959, 'setNo_1': '34